In [2]:
# Εγκατάσταση απαιτούμενων πακέτων
!pip install transformers==4.37.2 sentencepiece -q


[notice] A new release of pip available: 22.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
!pip install torch

     ------------------------------------ 216.1/216.1 MB 741.3 kB/s eta 0:00:00
     ---------------------------------------- 6.3/6.3 MB 2.4 MB/s eta 0:00:00
     -------------------------------------- 134.9/134.9 kB 7.8 MB/s eta 0:00:00
     ---------------------------------------- 1.7/1.7 MB 10.0 MB/s eta 0:00:00
     ------------------------------------- 536.2/536.2 kB 11.2 MB/s eta 0:00:00



[notice] A new release of pip available: 22.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
# Imports
import torch
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM

c:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
text1 = """Today is our dragon boat festival, in our Chinese culture, to celebrate it with all safe and great in our lives. Hope you too, to enjoy it as my deepest wishes.
Thank your message to show our words to the doctor, as his next contract checking, to all of us.
I got this message to see the approved message. In fact, I have received the message from the professor, to show me, this, a couple of days ago. I am very appreciated the full support of the professor, for our Springer proceedings publication."""

text2 = """During our final discuss, I told him about the new submission — the one we were waiting since last autumn, but the updates was confusing as it not included the full feedback from reviewer or maybe editor?
Anyway, I believe the team, although bit delay and less communication at recent days, they really tried best for paper and cooperation. We should be grateful, I mean all of us, for the acceptance and efforts until the Springer link came finally last week, I think.
Also, kindly remind me please, if the doctor still plan for the acknowledgments section edit before he sending again. Because I didn’t see that part final yet, or maybe I missed, I apologize if so.
Overall, let us make sure all are safe and celebrate the outcome with strong coffee and future targets"""

In [7]:
# 1. Grammar/Fluency Fix (Flan-T5 small custom prompt)
flan_tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-small")
flan_model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-small")

def flan_correct(text):
    prompt = "Correct the grammar and make the following text clear and natural:\n" + text
    inputs = flan_tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512)
    outputs = flan_model.generate(**inputs, max_length=512)
    return flan_tokenizer.decode(outputs[0], skip_special_tokens=True)

recon1_text1 = flan_correct(text1)
recon1_text2 = flan_correct(text2)

c:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
c:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\user\.cache\huggingface\hub\models--google--flan-t5-small. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Develo

In [8]:
# 2. T5 Paraphrase (Vamsi/T5_Paraphrase_Paws)
t5_model = AutoModelForSeq2SeqLM.from_pretrained("Vamsi/T5_Paraphrase_Paws")
t5_tokenizer = AutoTokenizer.from_pretrained("Vamsi/T5_Paraphrase_Paws")

def t5_paraphrase(text):
    input_text = "paraphrase: " + text + " </s>"
    encoding = t5_tokenizer.encode_plus(input_text, return_tensors="pt", max_length=512, truncation=True)
    output = t5_model.generate(
        **encoding,
        max_length=256,
        num_beams=5,
        num_return_sequences=1,
        temperature=1.5
    )
    return t5_tokenizer.decode(output[0], skip_special_tokens=True)

recon2_text1 = t5_paraphrase(text1)
recon2_text2 = t5_paraphrase(text2)

c:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\user\.cache\huggingface\hub\models--Vamsi--T5_Paraphrase_Paws. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenizati

ImportError: 
T5Converter requires the protobuf library but it was not found in your environment. Checkout the instructions on the
installation page of its repo: https://github.com/protocolbuffers/protobuf/tree/master/python#installation and follow the ones
that match your environment. Please note that you may need to restart your runtime after installation.


In [9]:
# 3. BART (facebook/bart-large-cnn) for semantic simplification
bart = pipeline("text2text-generation", model="facebook/bart-large-cnn")

recon3_text1 = bart(text1, max_length=256, truncation=True)[0]["generated_text"]
recon3_text2 = bart(text2, max_length=256, truncation=True)[0]["generated_text"]


c:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\user\.cache\huggingface\hub\models--facebook--bart-large-cnn. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling bac

: 

In [1]:
# Εμφάνιση Αποτελεσμάτων Μας
print("\n========== RECONSTRUCTION RESULTS ==========\n")

print("=== 1. FLAN Grammar Correction ===\n")
print("Text 1:\n", recon1_text1, "\n")
print("Text 2:\n", recon1_text2, "\n")

print("\n=== 2. T5 Paraphraser ===\n")
print("Text 1:\n", recon2_text1, "\n")
print("Text 2:\n", recon2_text2, "\n")

print("\n=== 3. BART Summary/Rewrite ===\n")
print("Text 1:\n", recon3_text1, "\n")
print("Text 2:\n", recon3_text2, "\n")


========== RECONSTRUCTION RESULTS ==========

=== 1. FLAN Grammar Correction ===



NameError: name 'recon1_text1' is not defined